In [ ]:
%matplotlib inline
import os
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
import psycopg2
from dotenv import load_dotenv

In [ ]:
print(f"Importando do arquivo .env: {load_dotenv(dotenv_path=r'../.env')}")

In [ ]:
#https://www.youtube.com/watch?v=MEEVcpnfElY
leituras_dique6c = pd.read_csv('../arquivos/leituras_dique6c.csv')
leituras_bvsa = pd.read_csv('../arquivos/leituras_bvsa.csv')

In [ ]:
loc_bvsa = pd.read_csv('../arquivos/loc_bvsa.csv', sep=';', decimal=',')
loc_dique6c = pd.read_csv('../arquivos/loc_dique6c.csv', sep=';', decimal=',')

In [ ]:
loc_sensores = pd.read_excel('../arquivos/LOCALIZAÇÃO_SENSORES.xlsx', engine='openpyxl', header=2)

In [ ]:
leituras_bvsa['area'] = 'BVSA'
leituras_bvsa['area'] = 'Dique 6C'

In [ ]:
leituras = pd.concat([leituras_bvsa, leituras_dique6c])
leituras = leituras.rename(columns={"Sensor":"sensor", "Data":"data", "Valor":"valor", "Unidade":"unidade", "Direcao_Saida":"direcao_saida", "Estado":"estado"})
leituras['direcao_saida'] = leituras['direcao_saida'].replace('Cota NA', 'Cota do NA')

In [ ]:
leituras_bvsa['Sensor'] = leituras_bvsa['Sensor'].apply(lambda x: x.replace("(I)", '').replace("(S)", '').strip())
leituras_dique6c['Sensor'] = leituras_dique6c['Sensor'].apply(lambda x: x.replace("(I)", '').replace("(S)", '').strip())
loc_bvsa['Sensor'] = loc_bvsa['Sensor'].apply(lambda x: x.replace("(I)", '').replace("(S)", '').strip())
loc_dique6c['Sensor'] = loc_dique6c['Sensor'].apply(lambda x: x.replace("(I)", '').replace("(S)", '').strip())

In [ ]:
leituras[leituras['direcao_saida'] == 'Cota do NA']['valor'].mean()

In [ ]:
leituras['direcao_saida'].value_counts()

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

#leituras['id_sensor'] = le.fit_transform(leituras['sensor'])
leituras['id_unidade'] = le.fit_transform(leituras['unidade']) +1
leituras['id_direcao_saida'] = le.fit_transform(leituras['direcao_saida']) +1
leituras['id_estado'] = le.fit_transform(leituras['estado']) +1
leituras['id_area'] = le.fit_transform(leituras['area']) + 1

leituras = leituras.reset_index(drop=True)

In [ ]:
#Criação da tabela de sensores

In [ ]:
aux = pd.concat([leituras_bvsa[['Sensor']], leituras_dique6c[['Sensor']], loc_bvsa[['Sensor']], loc_dique6c[['Sensor']]])
tabela_sensores = aux.drop_duplicates().sort_values(by=['Sensor']).reset_index(drop=True)
tabela_sensores['Sensor'] = tabela_sensores['Sensor'].apply(lambda x: x.replace("‐", '-').strip())
tabela_sensores.index = tabela_sensores.index+1

In [ ]:
tabela_sensores.to_excel('../tabelas/sensores.xlsx')

In [ ]:
dict_sensores = tabela_sensores.reset_index()[['Sensor','index']].to_dict(orient = 'tight')
dict_sensores = dict(dict_sensores['data']);

In [ ]:
leituras['id_sensor'] = leituras['sensor'].map(dict_sensores) +1

In [ ]:
leituras = leituras[['id_sensor','id_unidade', 'id_direcao_saida', 'id_estado', 'id_area', 'sensor', 'data', 'valor', 'unidade', 'direcao_saida', 'estado', 'area']]

In [ ]:
sensores = pd.DataFrame(tabela_sensores)
sensores = sensores.rename(columns={"Sensor":"name"})
sensores

In [ ]:
#Criação da Tabela de Unidade

In [ ]:
tabela_unidades = pd.concat([leituras_bvsa['Unidade'], leituras_dique6c['Unidade']])
tabela_unidades = tabela_unidades.drop_duplicates().sort_values().reset_index(drop=True)
unidades = pd.DataFrame(tabela_unidades)
unidades = unidades.rename(columns={"Unidade":"name"})
unidades

In [ ]:
#Tabela de Direcao Saida

In [ ]:
tabela_direcao_saida = pd.concat([leituras_bvsa['Direcao_Saida'], leituras_dique6c['Direcao_Saida']])
tabela_direcao_saida = tabela_direcao_saida.drop_duplicates().sort_values().reset_index(drop=True)
direcao_saida = pd.DataFrame(tabela_direcao_saida)
direcao_saida = direcao_saida.rename(columns={"Direcao_Saida":"name"})
direcao_saida

In [ ]:
# Tabela de Estado
leituras['estado'].value_counts()

In [ ]:
# Leituras duplicadas
#leituras[leituras.duplicated(keep=False)]
leituras = leituras.drop_duplicates()

In [ ]:
#Tratamento dos dados de localização
import geopandas as gpd

In [ ]:
localizacao = gpd.GeoDataFrame(pd.concat([loc_bvsa, loc_dique6c]))
localizacao = localizacao.rename(columns={"Sensor":"sensor", "Latitude":"latitude", "Longitude":"longitude", "Altura":"altura"})

In [ ]:
localizacao['sensor'] = localizacao['sensor'].apply(lambda x: x.replace("‐", '-').strip())

In [ ]:
localizacao['id_sensor'] = localizacao['sensor'].map(dict_sensores)

In [ ]:
localizacao

In [ ]:
x_latitude = localizacao['latitude']
y_longitude = localizacao['longitude']
z_altura = localizacao['altura']

In [ ]:
localizacao['geom'] = gpd.GeoSeries.from_xy(x =y_longitude, y=x_latitude, z=z_altura, crs='EPSG:4326')

In [ ]:
localizacao = localizacao[['id_sensor', 'sensor', 'latitude', 'longitude', 'altura', 'geom']]

In [ ]:
localizacao.head()

In [ ]:
SGBD_HOMOLOGACAO =  os.environ.get("DATABASE_URL")
HOST_HOMOLOGACAO =  os.environ.get("PGHOST")
USER_HOMOLOGACAO = os.environ.get("PGUSER")
DATABASE_HOMOLOGACAO =  os.environ.get("PGDATABASE")
PORT_HOMOLOGACAO =  os.environ.get("PGPORT")
PASSWORD_HOMOLOGACAO = os.environ.get("PGPASSWORD")
SQLALCHEMY_HOMOLOGACAO = os.environ.get("DATABASE_URL")

In [ ]:
print("Criando conexão com o banco.")
try: 
    engine = create_engine(SQLALCHEMY_HOMOLOGACAO)
    conn = engine.connect()
    print('Conexão com banco de dados criada...')
except:
    print('Falha ao criar conexão com banco de dados.')

In [ ]:
localizacao = localizacao.set_geometry('geom')

In [ ]:
localizacao_ = localizacao[['id_sensor', 'sensor', 'latitude', 'longitude', 'altura']]

In [ ]:
sensores.to_sql('sensores', schema='pesquisa', con=conn, if_exists='replace', index=False)
unidades.to_sql('unidades', schema='pesquisa', con=conn, if_exists='replace', index=False)
direcao_saida.to_sql('direcao_saida', schema='pesquisa', con=conn, if_exists='replace', index=False)
leituras.to_sql('leituras', schema='pesquisa', con=conn, if_exists='replace', index=False)
localizacao.to_postgis('localizacao', schema='pesquisa', con=conn, if_exists='replace', index=False)